In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from pathlib import Path

Inactive orgs

In [2]:
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?Status=Inactive&Roles=RO197,RO98&Limit=1000"
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())
  
# print the json response
#print(data_json)

In [3]:
inactive_orgs = []

for q in data_json['Organisations']:
    inactive_orgs.append({
        "ORG_NAME": q['Name'],
        "ORG_CODE" : q['OrgId']
    })

In [4]:
with open("./tempdir/inactive_organisations.txt", 'w') as f:
    json.dump(inactive_orgs, f)
f.close()


In [5]:
inactive_orgs = pd.read_json('./tempdir/inactive_organisations.txt')

Load turnover data

In [6]:
path = "tempdir/turnover"  # or unix / linux / mac path

# Get the files from the path provided in the OP
files_annual = Path(path).glob('*annual.csv')
files_monthly = Path(path).glob('*monthly.csv')

columns_to_merge = {
    'Period' : 'PERIOD',
    'Type' : 'TYPE',
    'Org code': 'ORG_CODE',
    'Org name': 'ORG_NAME',
    'NHSE region code': 'NHSE_REGION_CODE',
    'NHSE region name': 'NHSE_REGION_NAME',
    'ICS code': 'ICS_CODE',
    'ICS name': 'ICS_NAME',
    'Cluster group': 'CLUSTER_GROUP',
    'Benchmark group': 'BENCHMARK_GROUP',   
    'Staff group': 'STAFF_GROUP',
    'HC':'HC',
    'FTE':'FTE'
    # 'Org Code': 'ORG_CODE',
    # 'Org Name': 'ORG_NAME',
    # 'Org Type': 'ORG_TYPE',
    # 'FTE days lost': 'FTE_DAYS_LOST',
    # 'FTE Days Sick' : 'FTE_DAYS_LOST',
    # 'FTE days available': 'FTE_DAYS_AVAILABLE',
    # 'FTE Days Available' : 'FTE_DAYS_AVAILABLE',
    # 'Sickness absence rate (%)': 'SICKNESS_ABSENCE_RATE_PERCENT',

}

In [7]:
# for f in files_annual:
#     print(f.as_posix())

Concat files in tempdir

In [8]:
dfs = list()
df = None

for f in files_annual:
    #print(f.as_posix())
    data = pd.read_csv(f.as_posix())
   # print(f"Number of rows: {len(data)}")
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file_date'] = str(f)[17:27] # Varies depending on name
    data.columns = [columns_to_merge.get(k,k) for k in data.columns]
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

In [9]:
list(df)

['PERIOD',
 'TYPE',
 'ORG_CODE',
 'ORG_NAME',
 'NHSE_REGION_CODE',
 'NHSE_REGION_NAME',
 'CLUSTER_GROUP',
 'BENCHMARK_GROUP',
 'STAFF_GROUP',
 'HC',
 'FTE',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 'Unnamed: 18',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Unnamed: 22',
 'file_date',
 'ICS_CODE',
 'ICS_NAME']

In [10]:
# remove regions, benchmarking etc. here and rejoin at later stage with latest ref table
# df1 = df[["file_date", "PERIOD", "ORG_CODE", "ORG_NAME", "NHSE_REGION_CODE", 
#           "NHSE_REGION_NAME", "CLUSTER_GROUP", "BENCHMARK_GROUP", "STAFF_GROUP", 
#           "TYPE", "HC", "FTE"]]

df1 = df[["file_date", "PERIOD", "ORG_CODE", "STAFF_GROUP", 
          "TYPE", "HC", "FTE"]]

In [11]:
df2 = df1.sort_values(by='file_date', 
                      ascending = False).groupby(['PERIOD', 'ORG_CODE',
                        'STAFF_GROUP', 'TYPE']).first().reset_index()

In [12]:
# set date columns to date types
# set leavers and joiners date to be the "from" date in original col
# correct date so that it is always first of the month
df2['DATE'] = df2.apply(lambda x: datetime.strptime(x['PERIOD'].split(' to ')[1]+'01', 
                                '%Y%m%d') 
                               if x['TYPE'] in ('Leavers', 
                                'Joiners') else (datetime.strptime('01/'+x['PERIOD'][3:10], 
                                                                   '%d/%m/%Y') 
                                                                            if x['TYPE'] == 'Denoms' else None), 
                                                                            axis = 1) 
df2['file_date'] = pd.to_datetime(df2['file_date'], yearfirst = True)

In [13]:
df3 = df2.groupby(['ORG_CODE', 'STAFF_GROUP', 'DATE']).apply(lambda x: 
        pd.Series({
        'n': x.shape[0],
        'join_HC': x.loc[x['TYPE'].str.contains('Joiners'), 'HC'].values[0] if any(x['TYPE'] == 'Joiners') else None,
        'join_FTE': x.loc[x['TYPE'] == 'Joiners', 'FTE'].values[0] if any(x['TYPE'] == 'Joiners') else None,
        'leave_HC': x.loc[x['TYPE'] == 'Leavers', 'HC'].values[0] if any(x['TYPE'] == 'Leavers') else None,
        'leave_FTE': x.loc[x['TYPE'] == 'Leavers', 'FTE'].values[0] if any(x['TYPE'] == 'Leavers') else None,
        'denom_HC': x.loc[x['TYPE'] == 'Denoms', 'HC'].values[0] if any(x['TYPE'] == 'Denoms') else None,
        'denom_FTE': x.loc[x['TYPE'] == 'Denoms', 'FTE'].values[0] if any(x['TYPE'] == 'Denoms') else None,
    })).reset_index()

# 10 min runtime MR work laptop

In [51]:
# org_ref = pd.read_csv('../REF_ORGANISATION.csv')

# # keep only first five columns
# org_ref = org_ref.iloc[:,0:5]

# # rename columns
# org_ref.columns = ['ORG_CODE', 'ORG_CODE_USE', 'ORG_NAME_LEGACY',
#                     'ORG_NAME', 'ORG_STATUS']

# # keep only org_code and org_status
# org_ref = org_ref[['ORG_CODE', 'ORG_STATUS']]

# org_ref.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21813 entries, 0 to 21812
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ORG_CODE    21813 non-null  object
 1   ORG_STATUS  21813 non-null  object
dtypes: object(2)
memory usage: 341.0+ KB


In [52]:
# # join org_ref to df3
# df4 = pd.merge(df3, org_ref, how='left', on=['ORG_CODE'])

# df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292545 entries, 0 to 292544
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   ORG_CODE     292545 non-null  object        
 1   STAFF_GROUP  292545 non-null  object        
 2   DATE         292545 non-null  datetime64[ns]
 3   n            292545 non-null  float64       
 4   join_HC      198679 non-null  float64       
 5   join_FTE     198679 non-null  float64       
 6   leave_HC     213642 non-null  float64       
 7   leave_FTE    213642 non-null  float64       
 8   denom_HC     267618 non-null  float64       
 9   denom_FTE    267618 non-null  float64       
 10  ORG_STATUS   291998 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(3)
memory usage: 24.6+ MB


In [53]:
# show unique org_status values
# df4['ORG_STATUS'].unique()

array(['Open', 'Closed', nan], dtype=object)

In [54]:
# drop organisations where org_status is not open
#df5 = df4[df4['ORG_STATUS'] == 'Open']

In [55]:
#inactive_orgs.info()

In [56]:
# Merge inactive dataframe
#df4 = pd.merge(df3, inactive_orgs, on='ORG_CODE', how='left')

In [57]:
#orgs_pre_filter = df4['ORG_CODE'].nunique()

In [41]:
# # # Filter out inactive organizations
# df4 = df4[~df4['ORG_NAME'].notna()].copy()
# df4.drop(columns='ORG_NAME', inplace=True)
# # how many orgs were filtered out?
# orgs_post_filter = df4['ORG_CODE'].nunique()
# n_orgs_filtered = orgs_pre_filter - orgs_post_filter
# print(f"Number of inactive organizations filtered out: {n_orgs_filtered}")

Number of inactive organizations filtered out: 130


In [112]:
# Merge with latest org list
# ref table with org information
url_ref_org = '../REF_ORGANISATION.csv'
ref_org = pd.read_csv(url_ref_org)

ref_org = ref_org.drop(['Org_Code_For_Join','Org_Open_Date',
                        'Org_Region_Code',
                        'Org_System_Code','Org_ICB_Name',
                        'Org_Close_Date', 'Org_Name','Org_Type','Org_Post_Code',
                        'Legacy_Org_Close_Date','UDALFileID','Org_System_Name',
                        ' NHS Provider flag ',' Total WTE recorded '],axis=1)

ref_org.rename(columns={'Org_Code_For_Use':'ORG_CODE','Org_Type_Grouped':'ORG_TYPE',
                        'Org_Name_For_Use':'ORG_NAME',
                        'Org_Region_Name':'region_name'},inplace=True)

ref_org.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21813 entries, 0 to 21812
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ORG_CODE     21813 non-null  object
 1   ORG_NAME     21813 non-null  object
 2   Org_Status   21813 non-null  object
 3   region_name  21647 non-null  object
 4   ORG_TYPE     21813 non-null  object
dtypes: object(5)
memory usage: 852.2+ KB


In [129]:
# org reference data merge
df5 = pd.merge(df3, ref_org, on=['ORG_CODE'],how='left')

In [130]:
df5.head()

,ORG_CODE,STAFF_GROUP,DATE,n,join_HC,join_FTE,leave_HC,leave_FTE,denom_HC,denom_FTE,ORG_NAME,Org_Status,region_name,ORG_TYPE
0,00D,All staff groups,2017-04-01,1.0,NaN,NaN,NaN,NaN,1.0,0.1,NaN,NaN,NaN,NaN
1,00D,All staff groups,2017-05-01,1.0,NaN,NaN,NaN,NaN,1.0,0.1,NaN,NaN,NaN,NaN
2,00D,All staff groups,2017-06-01,1.0,NaN,NaN,NaN,NaN,1.0,0.1,NaN,NaN,NaN,NaN
3,00D,All staff groups,2017-07-01,1.0,NaN,NaN,NaN,NaN,1.0,0.1,NaN,NaN,NaN,NaN
4,00D,All staff groups,2017-08-01,1.0,NaN,NaN,NaN,NaN,1.0,0.1,NaN,NaN,NaN,NaN


In [131]:
# drop where org status is not open
df5 = df5[df5['Org_Status'] == 'Open']

# drop org status column
df5 = df5.drop(['Org_Status'], axis=1)

# Drop ICBs
df5 = df5[~df5['ORG_TYPE'].isin(['INTEGRATED CARE BOARD'])]

# Drop NAN org types
df5 = df5.dropna(subset=['ORG_TYPE'])

# cut rows where denom_FTE is null or 0 - no staff in post for that group/org/period
df5 = df5[df5['denom_FTE'] != 0]
df5 = df5[df5['denom_FTE'].notna()]


In [132]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 223188 entries, 129414 to 368685
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   ORG_CODE     223188 non-null  object        
 1   STAFF_GROUP  223188 non-null  object        
 2   DATE         223188 non-null  datetime64[ns]
 3   n            223188 non-null  float64       
 4   join_HC      178819 non-null  float64       
 5   join_FTE     178819 non-null  float64       
 6   leave_HC     175224 non-null  float64       
 7   leave_FTE    175224 non-null  float64       
 8   denom_HC     223188 non-null  float64       
 9   denom_FTE    223188 non-null  float64       
 10  ORG_NAME     223188 non-null  object        
 11  region_name  223188 non-null  object        
 12  ORG_TYPE     223188 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 23.8+ MB


In [133]:
df6 = None
df6 = df5.copy()
df6.sort_values('DATE')
df6.columns

Index(['ORG_CODE', 'STAFF_GROUP', 'DATE', 'n', 'join_HC', 'join_FTE',
       'leave_HC', 'leave_FTE', 'denom_HC', 'denom_FTE', 'ORG_NAME',
       'region_name', 'ORG_TYPE'],
      dtype='object')

In [134]:
df6['denom_FTE_12'] = df6.sort_values(by=['DATE']).groupby(['ORG_CODE','STAFF_GROUP'])['denom_FTE'].shift(12)
df6['denom_HC_12'] = df6.sort_values(by=['DATE']).groupby(['ORG_CODE','STAFF_GROUP'])['denom_HC'].shift(12)

In [135]:
df6['denom_FTE_mean'] = df6[['denom_FTE', 'denom_FTE_12']].mean(axis=1)
df6['denom_HC_mean'] = df6[['denom_HC', 'denom_HC_12']].mean(axis=1)

In [143]:
df6.head()

,ORG_CODE,STAFF_GROUP,month_year,n,join_HC,join_FTE,leave_HC,leave_FTE,denom_HC,denom_FTE,ORG_NAME,region_name,ORG_TYPE,denom_FTE_12,denom_HC_12,denom_FTE_mean,denom_HC_mean
129414,R0A,All staff groups,2017-10-01,1.0,NaN,NaN,NaN,NaN,19130.0,17095.95832,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST,ACUTE,NaN,NaN,17095.95832,19130.0
129415,R0A,All staff groups,2017-10-01,1.0,NaN,NaN,NaN,NaN,19130.0,17095.95832,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST,ACUTE,NaN,NaN,17095.95832,19130.0
129416,R0A,All staff groups,2017-10-01,1.0,NaN,NaN,NaN,NaN,19130.0,17095.95832,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST,ACUTE,NaN,NaN,17095.95832,19130.0
129417,R0A,All staff groups,2017-11-01,1.0,NaN,NaN,NaN,NaN,19211.0,17171.25859,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST,ACUTE,NaN,NaN,17171.25859,19211.0
129418,R0A,All staff groups,2017-11-01,1.0,NaN,NaN,NaN,NaN,19211.0,17171.25859,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST,ACUTE,NaN,NaN,17171.25859,19211.0


In [137]:
# rename DATE column to month_year
df6.rename(columns={'DATE':'month_year'}, inplace=True)

In [138]:
#df5[(df5['ORG_NAME']=='Yorkshire Ambulance Service NHS Trust') & (df5['STAFF_GROUP']=='All staff groups') & (df5['DATE'].dt.month == 7)]

In [139]:
df7 = df6.copy()

In [140]:
df7 = df7.sort_values(by=['ORG_CODE','month_year'], ascending = False)

In [141]:
df7.to_csv('../annual_turnover.csv', index = False)

In [142]:
# save a copy to 03_dashboard/data
#df7.to_csv('../03_dashboard/data/annual_turnover.csv', index = False)